In [1]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


In [2]:
ruta = '../data/retail_sales_dataset.csv'
df = pd.read_csv(filepath_or_buffer=ruta, sep=',')
df.shape

(1000, 9)

## Parte 2: Preprocesamiento de Datos

### 1. Transformación de Columnas:
* Utilizar ColumnTransformer para aplicar transformaciones específicas a diferentes columnas.
* Realizar codificación de variables categóricas y escalado de variables numéricas.

In [3]:
# Definir las columnas numéricas y categóricas
numeric_cols = ['Age', 'Quantity', 'Price per Unit', 'Total Amount']
categorical_cols = ['Gender', 'Product Category']

# Definir las transformaciones: 
# - Escalado para las columnas numéricas
# - OneHotEncoding para las columnas categóricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),  # Escalar las columnas numéricas
        ('cat', OneHotEncoder(), categorical_cols) # OneHotEncoding para las columnas categóricas
    ])


In [4]:
# Aplicar las transformaciones
X_transformed = preprocessor.fit_transform(df)

# Mostrar el resultado transformado
print(X_transformed)


[[-0.54056476  0.42926498 -0.68512265 ...  1.          0.
   0.        ]
 [-1.12559156 -0.45399629  1.68846418 ...  0.          1.
   0.        ]
 [ 0.62948884 -1.33725757 -0.7906154  ...  0.          0.
   1.        ]
 ...
 [-1.34497661  1.31252625 -0.81698859 ...  1.          0.
   0.        ]
 [-0.39430806  0.42926498 -0.68512265 ...  0.          0.
   1.        ]
 [ 0.41010379  1.31252625 -0.7906154  ...  0.          0.
   1.        ]]


In [5]:
# Obtener nombres de las columnas después de OneHotEncoding
encoded_columns = preprocessor.transformers_[1][1].get_feature_names_out(categorical_cols)

# Reconstruir el DataFrame
df_transformed = pd.DataFrame(X_transformed, columns=numeric_cols + list(encoded_columns))

# Ver el DataFrame transformado
print(df_transformed)


          Age  Quantity  Price per Unit  Total Amount  Gender_Female  \
0   -0.540565  0.429265       -0.685123     -0.546704            0.0   
1   -1.125592 -0.453996        1.688464      0.971919            1.0   
2    0.629489 -1.337258       -0.790615     -0.761098            0.0   
3   -0.321180 -1.337258        1.688464      0.078611            0.0   
4   -0.833078 -0.453996       -0.685123     -0.636035            0.0   
..        ...       ...             ...           ...            ...   
995  1.507029 -1.337258       -0.685123     -0.725366            0.0   
996  0.775746  0.429265       -0.790615     -0.653901            0.0   
997 -1.344977  1.312526       -0.816989     -0.636035            1.0   
998 -0.394308  0.429265       -0.685123     -0.546704            1.0   
999  0.410104  1.312526       -0.790615     -0.600303            0.0   

     Gender_Male  Product Category_Beauty  Product Category_Clothing  \
0            1.0                      1.0                      

### 2. Pipelines:
* Crear pipelines para automatizar el preprocesamiento de datos y asegurar la reproducibilidad.

In [6]:
# Crear un Pipeline con el preprocesador
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

# Transformar los datos
df_transformed_pipeline = pipeline.fit_transform(df)

# Convertir la matriz resultante en un DataFrame
df_transformed_pipeline = pd.DataFrame(df_transformed_pipeline, columns=numeric_cols + list(encoded_columns))

# Mostrar el DataFrame transformado
print(df_transformed_pipeline)


          Age  Quantity  Price per Unit  Total Amount  Gender_Female  \
0   -0.540565  0.429265       -0.685123     -0.546704            0.0   
1   -1.125592 -0.453996        1.688464      0.971919            1.0   
2    0.629489 -1.337258       -0.790615     -0.761098            0.0   
3   -0.321180 -1.337258        1.688464      0.078611            0.0   
4   -0.833078 -0.453996       -0.685123     -0.636035            0.0   
..        ...       ...             ...           ...            ...   
995  1.507029 -1.337258       -0.685123     -0.725366            0.0   
996  0.775746  0.429265       -0.790615     -0.653901            0.0   
997 -1.344977  1.312526       -0.816989     -0.636035            1.0   
998 -0.394308  0.429265       -0.685123     -0.546704            1.0   
999  0.410104  1.312526       -0.790615     -0.600303            0.0   

     Gender_Male  Product Category_Beauty  Product Category_Clothing  \
0            1.0                      1.0                      

In [7]:
# Definir características (X) y etiqueta (y) (suponiendo que 'Total Amount' es la etiqueta)
X = df.drop(columns=['Transaction ID', 'Date', 'Customer ID', 'Total Amount'])
y = df['Total Amount']


In [8]:
# Definir las columnas numéricas y categóricas
numeric_cols = ['Age', 'Quantity', 'Price per Unit']
categorical_cols = ['Gender', 'Product Category']


In [9]:
# Definir el preprocesador para columnas numéricas y categóricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),  # Imputación de valores faltantes
            ('scaler', StandardScaler())  # Escalado
        ]), numeric_cols),
        
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputación de valores faltantes
            ('onehot', OneHotEncoder(handle_unknown='ignore'))  # Codificación OneHot
        ]), categorical_cols)
    ])


In [10]:
# Crear un pipeline que primero preprocesa los datos y luego entrena el modelo
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])


In [11]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


In [12]:
# Entrenar el pipeline en los datos de entrenamiento
pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Quantity',
                                                   'Price per Unit']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Gender',
                                                   'Product Category'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [13]:
# Evaluar el modelo en el conjunto de prueba
score = pipeline.score(X_test, y_test)
print(f"Accuracy en el conjunto de prueba: {score:.4f}")


Accuracy en el conjunto de prueba: 0.9960


In [14]:
# Nuevos datos para hacer predicciones
new_data = pd.DataFrame({
    'Gender': ['Male', 'Female'],
    'Age': [29, 42],
    'Product Category': ['Electronics', 'Clothing'],
    'Quantity': [1, 3],
    'Price per Unit': [150, 220]
})

# Hacer predicciones con el pipeline
predictions = pipeline.predict(new_data)
print(f"Predicciones: {predictions}")


Predicciones: [ 50 900]
